In [1]:
from mp_api.client import MPRester
import csv
from api_key import APIKEY
API_KEY = APIKEY

In [2]:
csv_file_path = 'mpids.csv'
data = []

with open(csv_file_path, mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        data.append(row)

MPID_ids_list = [nested_id[0] for nested_id in data]

In [3]:
materials = MPID_ids_list[2500:3000]
bandstruct = []
failed_ids = []

with MPRester(API_KEY) as mpr:
    for mp_id in materials:
        try:
            band_structure = mpr.get_bandstructure_by_material_id(mp_id)
            bandstruct.append(band_structure)
        except Exception as e:
            print(f"Failed to retrieve band structure for {mp_id}: {e}")
            failed_ids.append(mp_id)

# Remove the failed IDs from the materials list
for id in failed_ids:
    materials.remove(id)


Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Failed to retrieve band structure for mp-1195978: No setyawan_curtarolo band structure data found for mp-1195978


...

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Failed to retrieve band structure for mp-1197947: No setyawan_curtarolo band structure data found for mp-1197947


Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Failed to retrieve band structure for mp-779287: No setyawan_curtarolo band structure data found for mp-779287


In [4]:
print(len(materials))

332


In [5]:
mp_entries = []

with MPRester(API_KEY) as mpr:
    for mp_id in materials:
        material_entries = mpr.materials.search(material_ids=[mp_id])
        
        if material_entries:
            mp_entries.append(material_entries[0])

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

...

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
print(len(mp_entries),len(bandstruct))

332 332


In [7]:
chemsys_list = [entry.chemsys for entry in mp_entries if hasattr(entry, 'chemsys')]
composition_list = [entry.composition_reduced for entry in mp_entries if hasattr(entry, 'composition_reduced')] 
materials_gga = [material + "-GGA" for material in materials]
materials_id = materials

In [8]:
bandgaps = []

for i in bandstruct:
    bandgap = i.get_band_gap()
    bandgapenergy = bandgap.get('energy')
    bandgaps.append(bandgapenergy)

In [9]:
electroneg = []

for i in composition_list:
    electroneg.append(i.average_electroneg)

In [10]:
cbm = []
vbm = []

for i,x in zip(bandgaps,electroneg):

    bandcorrection = (i + 0.85)/2
    cbm_pre = x + bandcorrection
    vbm_pre = x - bandcorrection
    cbm_minusvac = cbm_pre - 4.5
    vbm_minusvac = vbm_pre - 4.5

    cbm.append(cbm_minusvac)
    vbm.append(vbm_minusvac)

In [11]:
band_edge = list(zip(cbm, vbm, materials))

In [12]:
CO_reduction_met = [item for item in band_edge if item[0] >= 0.12]
CHOH_reduction_met = [item for item in band_edge if 0 <= item[0] < 0.12]
CH4_reduction_met = [item for item in band_edge if -0.17 <= item[0] < 0]

In [13]:
print("CO:",len(CO_reduction_met),"||","CHOH:",len(CHOH_reduction_met),"||","CH4:",len(CH4_reduction_met))

CO: 4 || CHOH: 4 || CH4: 13


In [14]:
print(len(failed_ids))

168


In [15]:
#print("CO:",CO_reduction_met)
CO_mpids = [item[2] for item in CO_reduction_met]
print(CO_mpids)

['mp-532810', 'mp-12442', 'mp-554835', 'mp-3817']


In [16]:
#print("CHOH:",CHOH_reduction_met)
CHOH_mpids = [item[2] for item in CHOH_reduction_met]
print(CHOH_mpids)

['mp-7979', 'mp-557441', 'mp-558116', 'mp-554888']


In [17]:
#print("CH4:",CH4_reduction_met)
CH4_mpids = [item[2] for item in CH4_reduction_met]
print(CH4_mpids)

['mp-556518', 'mp-1091363', 'mp-553996', 'mp-7387', 'mp-20076', 'mp-20968', 'mp-735586', 'mp-7984', 'mp-8013', 'mp-545469', 'mp-557257', 'mp-560997', 'mp-644015']


In [18]:
#cbms_CO = [item[0] for item in CO_reduction_met]
#vbms_CO = [item[1] for item in CO_reduction_met]
#mps_CO = [item[2] for item in CO_reduction_met]

#cbms_CHOH = [item[0] for item in CHOH_reduction_met]
#vbms_CHOH = [item[1] for item in CHOH_reduction_met]
#mps_CHOH = [item[2] for item in CHOH_reduction_met]

In [19]:
'''
import matplotlib.pyplot as plt

water_oxidation = -1.23
co_reduction = 0.12

plt.figure(figsize=(6, 4))

for i in range(len(CO_reduction_met)):
    plt.hlines(cbms_above[i], i - 0.1, i + 0.1, colors='blue', label='CBM' if i == 0 else "")
    plt.hlines(vbms_above[i], i - 0.1, i + 0.1, colors='red', label='VBM' if i == 0 else "")

plt.axhline(y=water_oxidation, color='green', linestyle='--', label='Water Oxidation')
plt.axhline(y=co_reduction, color='green', linestyle='--', label='CO2 Redox')

# Adding labels and title
plt.ylabel('Energy (eV)')
plt.title('Band Gap Alignment: Above CO Reduction')
plt.legend()

# Show the plot
plt.show()

'''


'\nimport matplotlib.pyplot as plt\n\nwater_oxidation = -1.23\nco_reduction = 0.12\n\nplt.figure(figsize=(6, 4))\n\nfor i in range(len(CO_reduction_met)):\n    plt.hlines(cbms_above[i], i - 0.1, i + 0.1, colors=\'blue\', label=\'CBM\' if i == 0 else "")\n    plt.hlines(vbms_above[i], i - 0.1, i + 0.1, colors=\'red\', label=\'VBM\' if i == 0 else "")\n\nplt.axhline(y=water_oxidation, color=\'green\', linestyle=\'--\', label=\'Water Oxidation\')\nplt.axhline(y=co_reduction, color=\'green\', linestyle=\'--\', label=\'CO2 Redox\')\n\n# Adding labels and title\nplt.ylabel(\'Energy (eV)\')\nplt.title(\'Band Gap Alignment: Above CO Reduction\')\nplt.legend()\n\n# Show the plot\nplt.show()\n\n'

In [20]:
'''
water_oxidation = -1.23
co_reduction = 0.12

plt.figure(figsize=(6, 4))

for i in range(len(CO_reduction_below)):
    plt.hlines(cbms_below[i], i - 0.1, i + 0.1, colors='blue', label='CBM' if i == 0 else "")
    plt.hlines(vbms_below[i], i - 0.1, i + 0.1, colors='red', label='VBM' if i == 0 else "")

plt.axhline(y=water_oxidation, color='green', linestyle='--', label='Water Oxidation')
plt.axhline(y=co_reduction, color='green', linestyle='--', label='CO2 Redox')

# Adding labels and title
plt.ylabel('Energy (eV)')
plt.title('Band Gap Alignment: Below CO Reduction')
plt.legend()

# Show the plot
plt.show()
'''

'\nwater_oxidation = -1.23\nco_reduction = 0.12\n\nplt.figure(figsize=(6, 4))\n\nfor i in range(len(CO_reduction_below)):\n    plt.hlines(cbms_below[i], i - 0.1, i + 0.1, colors=\'blue\', label=\'CBM\' if i == 0 else "")\n    plt.hlines(vbms_below[i], i - 0.1, i + 0.1, colors=\'red\', label=\'VBM\' if i == 0 else "")\n\nplt.axhline(y=water_oxidation, color=\'green\', linestyle=\'--\', label=\'Water Oxidation\')\nplt.axhline(y=co_reduction, color=\'green\', linestyle=\'--\', label=\'CO2 Redox\')\n\n# Adding labels and title\nplt.ylabel(\'Energy (eV)\')\nplt.title(\'Band Gap Alignment: Below CO Reduction\')\nplt.legend()\n\n# Show the plot\nplt.show()\n'